In [7]:
from ttsxai.utils import jupyter_utils
# jupyter_utils.register_always_reload(packages=[
#     # 'ttsxai.interfaces',
#     # 'ttsxai.models.text2mel',
#     # 'ttsxai.models.mel2wave',
#     # 'ttsxai.utils',
# ])
jupyter_utils.register_always_reload(modules=[
    # 'ttsxai.text',
    # 'ttsxai.models.text2mel.tacotron2',
    # 'ttsxai.models.text2mel.fastspeech2',
    'ttsxai.probe.linear_probe',
    # 'ttsxai.pitch_calculator',
    # 'ttsxai.energy_calculator'
])

In [1]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt

import ttsxai


In [2]:
# import os
# import numpy as np
# from tqdm import tqdm
# from multiprocessing import Pool, cpu_count

# import torch
# import torch.nn as nn
# import torch.nn.functional as F

# import neurox.interpretation.utils as utils
# import neurox.interpretation.ablation as ablation
# import neurox.interpretation.linear_probe as linear_probe

# from ttsxai.utils.utils import read_ljs_metadata

In [3]:
save_dir = os.path.join(
    ttsxai.PACKAGE_DIR, 
    "data_df/LJSpeech/tacotron2_waveglow"
)

In [4]:
df_train = pd.read_pickle(os.path.join(save_dir, 'activation_df_train.pkl'))
df_val = pd.read_pickle(os.path.join(save_dir, 'activation_df_val.pkl'))

In [5]:
X_train = np.stack(df_train['activations'].to_numpy()).astype(np.float32)
y_train = df_train['duration'].to_numpy().astype(np.float32)
X_val = np.stack(df_val['activations'].to_numpy()).astype(np.float32)
y_val = df_val['duration'].to_numpy().astype(np.float32)

# ignore invalid data
X_train = X_train[y_train > 0]
y_train = y_train[y_train > 0]
X_val = X_val[y_val > 0]
y_val = y_val[y_val > 0]

# we predict log duration
y_train = np.log(y_train)
y_val = np.log(y_val)

In [8]:
from ttsxai.probe.linear_probe import LinearProbe

In [9]:
probe = LinearProbe(X_train.shape[1], 1)

In [73]:
X_train.dtype

dtype('float32')

In [7]:
probe = linear_probe.train_linear_regression_probe(
    X_train, y_train, lambda_l1=0.001, lambda_l2=0.001, num_epochs=1, probe=probe)

scores_train = linear_probe.evaluate_probe(probe, X_train, y_train, metric='mse')
print( '[* train score]', scores_train)
scores_test = linear_probe.evaluate_probe(probe, X_test, y_test, metric='mse')
print( '[* test score]', scores_test)

# scores['train'].append(scores_train['__OVERALL__'])
# scores['test'].append(scores_test['__OVERALL__'])

Training regression probe
Creating model...
Number of training instances: 851403


epoch [1/1]: 0it [00:00, ?it/s]

Epoch: [1/1], Loss: 0.0031


Evaluating: 0it [00:00, ?it/s]

Score (mse) of the probe: 0.09
[* train score] {'__OVERALL__': 0.08582296}


NameError: name 'X_test' is not defined

In [19]:
np.sum(Y_train == 0)

7641

In [14]:
X_train.shape

(859044,)